In [1]:
import os
import copy
import numpy as np
import pandas as pd
from tqdm import tqdm, trange

import torch
from torch_geometric.nn import GAE
from torch_geometric.utils import is_undirected, to_undirected


from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.preprocessing import MinMaxScaler

from model.GCN_GRU import GCN_GRU_singleGCN, Decoder
from utils import *

In [16]:
shop_col = 'stonc_6_label'
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

embedding_dim = 64
entity_dim = 8
epochs = 1000
learning_rate = 0.01
batch_size = 1000

weights_path = 'SingleGCN_GRU_Link_Prediction_without_newf'

In [3]:
data_path = './data'

chid_dict_file = 'sample/sample_50k_idx_map.npy'
cdtx_file = 'sample/sample_50k_cdtx.csv'
cust_file = 'preprocessed/df_cust_log_without_shop.csv'

chid_path = os.path.join(data_path, chid_dict_file)
cdtx_path = os.path.join(data_path, cdtx_file)
cust_path = os.path.join(data_path, cust_file)

In [4]:
df_cdtx, df_cust, n_users, n_shops = read_sample_files(cdtx_path,
                                                       cust_path,
                                                       chid_path,
                                                       shop_col)

Start reading cdtx file...
Finish reading cdtx file !
Start reading cust file...
Finish reading cust file !
Start maping encodeing...
Finish !!


In [5]:
shopping_y = (df_cust[['objam']].to_numpy() > 0 ).astype(int)

In [6]:
list_months = sorted(df_cdtx.csmdt.unique())
ma = df_cdtx.groupby(['chid', 'csmdt']).objam.count().max()

ignore_cols = ['chid', 'data_dt']
category_cols = [f'category_{i+1}' for i in range(6)]
numeric_cols = list(set(df_cust.columns) - set(category_cols) - set(ignore_cols))

emb_dims = list(df_cust[category_cols].nunique())

input_dim = len(category_cols)*entity_dim + len(numeric_cols)

In [7]:
edge_dict = {}
edge_weights_dict = {}
for month in tqdm(list_months):
    edges = df_cdtx[df_cdtx.csmdt==month].groupby(['chid', shop_col]).objam.sum()
    edge_pairs = np.stack([np.array(i) for i in edges.index]).T
    edge_pairs = torch.LongTensor(edge_pairs)
    
    edge_weights = np.log(edges.values+1)
    edge_weights = torch.Tensor(edge_weights)
    
    if not is_undirected(edge_pairs):
        edge_pairs = torch.cat([edge_pairs, edge_pairs[[1,0],:]], -1)
        edge_weights = edge_weights.repeat(2)
    
    edge_dict[month] = edge_pairs
    edge_weights_dict[month] = edge_weights

100%|██████████| 25/25 [00:18<00:00,  1.33it/s]


In [9]:
model = GAE(GCN_GRU_singleGCN(input_dim, embedding_dim, entity_dim, emb_dims), Decoder(embedding_dim, 1)).to(device)

x_num = []
x_cat = []
y = []
for i in list_months:
    cust_num_features = df_cust[df_cust.data_dt==i][numeric_cols].to_numpy()
    cust_num_features = torch.Tensor(cust_num_features)
    cust_cat_features = df_cust[df_cust.data_dt==i][category_cols].to_numpy()
    cust_cat_features = torch.LongTensor(cust_cat_features)
    
    shop_num_features = torch.zeros(n_shops, cust_num_features.shape[1])
    shop_cat_features = torch.zeros(n_shops, cust_cat_features.shape[1]).long()
    
    temp_y = shopping_y[df_cust.data_dt==i]
    x_num.append(torch.cat([cust_num_features, shop_num_features], 0).to(device))
    x_cat.append(torch.cat([cust_cat_features, shop_cat_features], 0).to(device))
    y.append(torch.Tensor(temp_y).to(device))
    
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
pos_edge_index = [i.to(device) for i in edge_dict.values()]
pos_edge_weigts = [i.to(device) for i in edge_weights_dict.values()]

In [12]:
model.encoder.load_state_dict(torch.load(weights_path))
model.eval()

criterion = torch.nn.BCELoss()

In [17]:
def train():
    model.train()
    train_output = np.array([])
    train_y = np.array([])
    for i in tqdm(range(10)):
        
        train_dataset = TensorDataset(y[i+12])
        train_loader = DataLoader(dataset=train_dataset, shuffle=False, batch_size=batch_size)
        
        for j, true_y in enumerate(train_loader):
            optimizer.zero_grad()
            z = model.encode(x_cat[i:i+12], x_num[i:i+12], pos_edge_index[i:i+12], pos_edge_weigts[i:i+12])
            output = model.decode(z[j*batch_size:(j+1)*batch_size]).sigmoid()
            loss = criterion(output, true_y[0].float())
            train_output = np.concatenate([train_output, output.cpu().detach().numpy().reshape(-1)])
            train_y = np.concatenate([train_y, true_y[0].cpu().detach().numpy().reshape(-1)])
        
            loss.backward(retain_graph=True)
            optimizer.step()
        
            
    return loss, train_output, train_y

In [18]:
def test():
    model.eval()
    test_output = np.array([])
    test_y = np.array([])
    
    for i in range(10,12):

        with torch.no_grad():
            z = model.encode(x_cat[i:i+12], x_num[i:i+12], pos_edge_index[i:i+12], pos_edge_weigts[i:i+12])
            output = model.decode(z[:n_users]).sigmoid()
            test_output = np.concatenate([test_output, output.cpu().detach().numpy().reshape(-1)])
            test_y = np.concatenate([test_y, y[i+12].cpu().detach().numpy().reshape(-1)])
            
    return test_output, test_y

In [19]:
for epoch in range(epochs):
    loss, train_output, train_y  = train()
    
    test_output, test_y = test()
    
    train_AUC = roc_auc_score(train_y, train_output)
    test_AUC = roc_auc_score(test_y, test_output)
    train_F1 = f1_score(train_y, train_output>0.5)
    test_F1 = f1_score(test_y, test_output>0.5)
    
    print(f'epoch:{epoch+1}\ntrain AUC:{train_AUC},test AUC:{test_AUC}')
    print(f'epoch:{epoch+1}\ntrain F1:{train_F1},test F1:{test_F1}')
    

  0%|          | 0/10 [00:00<?, ?it/s]

epoch:1
train AUC:0.9098308742906798,test AUC:0.920179834865662
epoch:1
train F1:0.8888707288996972,test F1:0.895958125734759


  0%|          | 0/10 [00:00<?, ?it/s]

epoch:2
train AUC:0.9099808414447458,test AUC:0.9202934908689507
epoch:2
train F1:0.8889007838966637,test F1:0.8960498233091914


  0%|          | 0/10 [00:00<?, ?it/s]

...

epoch:22
train AUC:0.9131681944299204,test AUC:0.9234688597484751
epoch:22
train F1:0.8916677569245443,test F1:0.8991589594568163


  0%|          | 0/10 [00:00<?, ?it/s]

epoch:23
train AUC:0.913270565038433,test AUC:0.9235437335126343
epoch:23
train F1:0.891760568333685,test F1:0.8991470425919845


 20%|██        | 2/10 [00:31<02:07, 15.89s/it]


KeyboardInterrupt: 